# Model Building

This notebook will go over the process of testing different variations of models to find the parameters that best suit the model for this application. 

* Model Type
* Metric Evaluation
* Hyper Parameter Tuning 


In [1]:
import sys


In [2]:
!{sys.executable} -m pip install pymysql

In [3]:
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
from scipy.stats.stats import pearsonr
import pandas as pd
import numpy as np
from datetime import datetime
import statistics
import matplotlib.pyplot as plt
import pymysql
import config
import transformations

In [4]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)

#gather all historical data to build model
RideWaits = pd.read_sql_query("call DisneyDB.RideWaitQuery", conn)

#transform data for model bulding
RideWaits = transformations.transformData(RideWaits)


C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["MagicHourType"] = pd.Categorical(RideWaits["MagicHourType"])
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days


In [5]:
RideWaits.head()

,RideId,Date,Time,Wait,Name,OpeningDate,Tier,Location,IntellectualProp,Status,...,Weekend,CharacterExperience,inEMH,validTime,EMHDay,TimeSinceOpen,TimeSinceMidday,MagicHourType,MinutesSinceOpen,TimeSinceRideOpen
0,0,2018-05-07,12:30:00,35,Astro Orbiter,1995-02-25,minor_attraction,Tomorrowland,None,1,...,0,0,0,1,1,3,2,Night,210.0,8472
1,2,2018-05-07,12:30:00,45,Big Thunder Mountain Railroad,1980-09-23,headliner,Frontierland,None,1,...,0,0,0,1,1,3,2,Night,210.0,13740
2,3,2018-05-07,12:30:00,45,Buzz Lightyears Space Ranger Spin,1998-10-07,minor_attraction,Tomorrowland,Pixar,1,...,0,0,0,1,1,3,2,Night,210.0,7152
3,6,2018-05-07,12:30:00,40,Dumbo the Flying Elephant,1971-10-01,minor_attraction,Fantasyland,AnimatedClassic,1,...,0,0,0,1,1,3,2,Night,210.0,17020
4,7,2018-05-07,12:30:00,25,Enchanted Tales with Belle,2012-12-06,minor_attraction,Fantasyland,AnimatedClassic,1,...,0,0,0,1,1,3,2,Night,210.0,1978


The data frame looks quite different than in the prevsious exploratory analysis frame. Certain columns have been removed in an effort to consolidate and extract the vital information that has been seen to make a difference in Wait times. 

In [6]:
keyFeatures = ["Name","MagicHourType", "Tier", "IntellectualProp", "SimpleStatus", "ParkName", "DayOfWeek", "Weekend", "TimeSinceOpen","MinutesSinceOpen", "CharacterExperience", "TimeSinceMidday", "inEMH", "EMHDay"]


In [7]:
keyFeatures

['Name',
 'MagicHourType',
 'Tier',
 'IntellectualProp',
 'SimpleStatus',
 'ParkName',
 'DayOfWeek',
 'Weekend',
 'TimeSinceOpen',
 'MinutesSinceOpen',
 'CharacterExperience',
 'TimeSinceMidday',
 'inEMH',
 'EMHDay']

We have established to this point that this list of features will cause the most impact on wait times and give the most insight into the dataset. 
These can be broken into categories into the information we are trying to gain:
* Ride Characteristics
    * Name
    * Tier
    * IntellectualProp (Intellectual Property)
    * ParkName (Which park is this ride located in)
    * CharacterExperience (Is this a character experience)
* Time of Day Information
    * DayOfWeek
    * Weekend 
    * TimeSinceOpen (How many hours is it since the park opened that day)
    * TimeSinceMidday (How many hours is it in absolute value since 2pm)
    * inEMH (is this wait time in an Extra magic hour window)
    * EMHDay (is this day at the park an Extra Magic Hour Day)
    * MagicHourType (is this a extra magic morning or an extra magic night)
    * Minutes since park open. This and TimeSinceOpen may add unwanted variance. So I may remove TimeSinceOpen
* Weather
    * SimpleStatus
    
As the dataset grows and the more months and weather characteristics pile in, this list of usable features may expand. For example, temperature is not being included in this list as all the data gathered as of today has been from one week, and the temperature pattern adds no value. 

In [8]:
categoryColumns = RideWaits.select_dtypes(include = ['category']).columns
RideWaits["Name"] = pd.Categorical(RideWaits["Name"]).codes
for col in categoryColumns:
    RideWaits[col] = pd.Categorical(RideWaits[col]).codes


In [9]:
RideWaits[keyFeatures].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53949 entries, 0 to 55752
Data columns (total 14 columns):
Name                   53949 non-null int8
MagicHourType          53949 non-null int8
Tier                   53949 non-null int8
IntellectualProp       53949 non-null int8
SimpleStatus           53949 non-null int8
ParkName               53949 non-null int8
DayOfWeek              53949 non-null int64
Weekend                53949 non-null int64
TimeSinceOpen          53949 non-null int64
MinutesSinceOpen       53949 non-null float64
CharacterExperience    53949 non-null int64
TimeSinceMidday        53949 non-null int64
inEMH                  53949 non-null int64
EMHDay                 53949 non-null int64
dtypes: float64(1), int64(7), int8(6)
memory usage: 6.5 MB


In [10]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(RideWaits[keyFeatures], RideWaits["Wait"], test_size = .25, random_state = 1)

rf = RandomForestRegressor(random_state = 1)
rf.fit(train_x, train_y)
predictions = rf.predict(test_x)
rmseBase = metrics.mean_squared_error(predictions, test_y)**(1/2)
r2Base = metrics.r2_score(predictions, test_y)
varBase = metrics.explained_variance_score(predictions,test_y)
pearsoncorrBase = pearsonr(predictions, test_y)
perrorBase = abs(predictions - test_y)/test_y
accuracyBase = 1 - statistics.median(perrorBase)
errorBase = abs(predictions - test_y)
merrorBase = errorBase.mean()
medErrorBase = statistics.median(errorBase)

In [11]:
print("RMSE: " + str(rmseBase))
print("r2: "+ str(r2Base))
print("var:" + str(varBase))
print("pearsonCorr: "+ str(pearsoncorrBase))
print("Accuracy: " + str(accuracyBase))
print("Mean Error: " + str(merrorBase))
print("Median Error: " + str(medErrorBase))

RMSE: 10.5297419588
r2: 0.822561101765
var:0.822562810208
pearsonCorr: (0.91755591020216287, 0.0)
Accuracy: 0.85
Mean Error: 6.33241114383
Median Error: 3.5


By using the defaults of the Random forest regressor we can get some baseline statistics with this algorithm. We see we have a high correlation meaning that our predictions are following the proper trend of our data. Our accuracy for our base model is 85% and is not a bad start, we also see that both our mean and median wait time error values are under 10 minutes which is also a good place to start. We can now try some other modeling methods as well as tune the hyper parameters for our random forest 
model. 

We are going to start with attempting to tune the hyper parameters for this model

Our focus will be on the following parameters: 
* n_estimators
* max_features
* max_depth
* min_samples_split
* min_samples_leaf
* bootstrap

In [12]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10,110, num = 11)]
max_depth.append(None)
min_samples_split = [2,5,10]
min_samples_leaf = [1,2,4]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [13]:
rfNew = RandomForestRegressor()
rf_randomized = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose = 2, random_state = 1, n_jobs = -1)

In [14]:
rf_randomized.fit(train_x, train_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 27.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 55.2min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=1, refit=True,
          return_train_score=True, scoring=None, verbose=2)

In [15]:
rf_randomized.best_params_

{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 1800}

In [16]:
rf_random = rf_randomized.best_estimator_

In [17]:
predictions = rf_random.predict(test_x)
rmseRandom = metrics.mean_squared_error(predictions, test_y)**(1/2)
r2Random = metrics.r2_score(predictions, test_y)
varRandom = metrics.explained_variance_score(predictions,test_y)
pearsoncorrRandom = pearsonr(predictions, test_y)
perrorRandom = abs(predictions - test_y)/test_y
accuracyRandom = 1 - statistics.median(perrorRandom)
errorRandom = abs(predictions - test_y)
merrorRandom = errorRandom.mean()
medErrorRandom = statistics.median(errorRandom)

In [18]:
print("RMSE: " + str(rmseRandom))
print("r2: "+ str(r2Random))
print("var:" + str(varRandom))
print("pearsonCorr: "+ str(pearsoncorrRandom))
print("Accuracy: " + str(accuracyRandom))
print("Mean Error: " + str(merrorRandom))
print("Median Error: " + str(medErrorRandom))

RMSE: 10.0204624125
r2: 0.833626044924
var:0.833628497388
pearsonCorr: (0.92525807617302891, 0.0)
Accuracy: 0.842966624205
Mean Error: 6.16052199444
Median Error: 3.64464177566


We see a very slight increase in RMSE, mean error, and median Error, but see no increase in overall accuracy.

In [20]:
gridSearch = {'bootstrap': [True],
              'max_depth' : [60,70,80,90],
              'max_features': ['auto',6,7,8,9,10],
              'min_samples_leaf': [1,2,3,4],
              'min_samples_split': [4,5,6,7],
              'n_estimators' : [800,1200,1600,1800,2000]}

In [21]:
from sklearn.model_selection import GridSearchCV
rf = RandomForestRegressor()
grid_search_rf = GridSearchCV(estimator = rf, param_grid = gridSearch, cv = 3, n_jobs = -1, verbose = 2)

In [22]:
grid_search_rf.fit(train_x, train_y)

Fitting 3 folds for each of 1920 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 51.7min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 98.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 149.8min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 224.4min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 326.1min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 448.7min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 574.7min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 742.0min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 897.6min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed: 1093.5min
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed: 1280.5min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [60, 70, 80, 90], 'max_features': ['auto', 6, 7, 8, 9, 10], 'min_samples_leaf': [1, 2, 3, 4], 'min_samples_split': [4, 5, 6, 7], 'n_estimators': [800, 1200, 1600, 1800, 2000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [23]:
grid_search_rf.best_params_

{'bootstrap': True,
 'max_depth': 60,
 'max_features': 8,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 1800}

In [24]:
best_grid = grid_search_rf.best_estimator_

In [25]:
predictions = best_grid.predict(test_x)
rmseGrid = metrics.mean_squared_error(predictions, test_y)**(1/2)
r2Grid = metrics.r2_score(predictions, test_y)
varGrid = metrics.explained_variance_score(predictions,test_y)
pearsoncorrGrid = pearsonr(predictions, test_y)
perrorGrid = abs(predictions - test_y)/test_y
accuracyGrid = 1 - statistics.median(perrorGrid)
errorGrid = abs(predictions - test_y)
merrorGrid = errorGrid.mean()
medErrorGrid = statistics.median(errorGrid)

In [26]:
print("RMSE: " + str(rmseGrid))
print("r2: "+ str(r2Grid))
print("var:" + str(varGrid))
print("pearsonCorr: "+ str(pearsoncorrGrid))
print("Accuracy: " + str(accuracyGrid))
print("Mean Error: " + str(merrorGrid))
print("Median Error: " + str(medErrorGrid))

RMSE: 10.0066008223
r2: 0.832396931877
var:0.832397958874
pearsonCorr: (0.92545982565000462, 0.0)
Accuracy: 0.843614455923
Mean Error: 6.17400494897
Median Error: 3.63263172399


A slight increase in accuracy but negligible effects across the board in most other categories

## Cross validation
Using the internal cross validation packages we can compute scores without worrying about over fitting or tuning hyper parameters to specific data sets. This can cause a pseudo leakage of our testing data into the training set. We will start the cross validation with the random forest regressor we got from the best estimator from the grid search.

In [12]:
from sklearn.model_selection import cross_val_score

In [13]:
rf = RandomForestRegressor(bootstrap = True, max_depth = 50, max_features = 7, min_samples_leaf = 1, n_estimators = 500)

In [14]:
scores = cross_val_score(rf, RideWaits[keyFeatures], RideWaits["Wait"], cv = 10)

In [15]:
scores

array([ 0.90298668,  0.80734268,  0.75187512,  0.70698203,  0.69998131,
        0.6405473 ,  0.74681927,  0.60565567,  0.65903657,  0.74175662])

This is just a single metric, it is perhaps more useful to obtain a number of metrics for each of our cross validated models.

In [27]:
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
def cross_validation_metrics(df, key_cols, target, folds):
    df = df.dropna(how = 'any')
    X = df[key_cols]
    y = np.array(df[target])
    overall_rmse = []
    overall_accuracy = []
    overall_median_error = []
    overall_mean_error = []
    overall_r2 = []
    corr = []
    kf = KFold(n_splits = folds)
    i = 1
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        rf = RandomForestRegressor(n_estimators = 1800,max_features = 8, min_samples_split = 4, min_samples_leaf = 1,max_depth = 60)
        rf.fit(X_train, y_train)
        predictions = rf.predict(X_test)
        predictions = predictions[y_test>0]
        y_test = y_test[y_test>0]
        rmse = metrics.mean_squared_error(predictions, y_test)**(1/2)
        var = metrics.explained_variance_score(predictions,y_test)
        pearsoncorr = pearsonr(predictions, np.array(y_test))
        perror = abs(predictions - y_test)/y_test
        mperror = statistics.median(perror)
        error = abs(predictions - y_test)
        merror = error.mean()
        print("Fold " + str(i))
        print("RMSE: "+ str(rmse))
        print("Correlation: "+ str(pearsoncorr))
        print("Accuracy: " + str(1-mperror))
        print("Mean Error: "+ str(merror))
        print("Median Error: "+ str(statistics.median(error)))
        print("-------------------------")
        overall_rmse.append(rmse)
        overall_accuracy.append((1-mperror))
        overall_median_error.append(statistics.median(error))
        overall_mean_error.append(merror)
        #overall_r2.append(r2)
        corr.append(pearsoncorr[0])
        i = i+1
    
    return_dict = {
        'rmse': np.mean(overall_rmse),
        'accuracy': np.mean(overall_accuracy),
        'median_error': np.mean(overall_median_error),
        'mean_error' : np.mean(overall_mean_error),
        'correlation':np.mean(corr)
    }
    return return_dict


In [28]:
model_metrics = cross_validation_metrics(RideWaits, keyFeatures,"Wait",10)

Fold 1
RMSE: 4.0768071871
Correlation: (0.99086966862149617, 0.0)
Accuracy: 0.955849681183
Mean Error: 2.5506933953
Median Error: 1.40542438272
-------------------------
Fold 2
RMSE: 3.85614893798
Correlation: (0.99172386853000649, 0.0)
Accuracy: 0.957787220752
Mean Error: 2.4152382617
Median Error: 1.38825727513
-------------------------
Fold 3
RMSE: 13.2120515742
Correlation: (0.9017352216330411, 0.0)
Accuracy: 0.805769680936
Mean Error: 7.91203391906
Median Error: 4.2199537037
-------------------------
Fold 4
RMSE: 12.405944846
Correlation: (0.88888160849203757, 0.0)
Accuracy: 0.786774934363
Mean Error: 8.41775798993
Median Error: 5.53959475709
-------------------------
Fold 5
RMSE: 17.8680369258
Correlation: (0.80430034801130035, 0.0)
Accuracy: 0.727675443017
Mean Error: 11.4816431623
Median Error: 6.33975088183
-------------------------
Fold 6
RMSE: 14.2793373429
Correlation: (0.84975110690411437, 0.0)
Accuracy: 0.730025364759
Mean Error: 9.61863304535
Median Error: 6.24026775693


In [29]:
model_metrics

{'accuracy': 0.79643710970199355,
 'correlation': 0.8859910761747164,
 'mean_error': 8.2578994088581634,
 'median_error': 4.9529759599968104,
 'rmse': 12.58763456256524}

In [30]:
metric_frame = pd.DataFrame(list(model_metrics.items()), columns = ['Metric Name', 'Metric Value'])

In [31]:
metric_frame

,Metric Name,Metric Value
0,rmse,12.587635
1,accuracy,0.796437
2,median_error,4.952976
3,mean_error,8.257899
4,correlation,0.885991
